## Project 1.2 - Test Functions

Group: Emmy Noether

Students: Janik Rausch (628334), Camilo Tello Breuer (633291), Ida Wöstheinreich (628428)

In [1]:
using LinearAlgebra,Random,Printf
include("Ground_state.jl")

ground_state

**Tests of the Hamiltonian**

The Hamiltonian must be linear, hermitian and positive-definite. Also, plane waves must be eigenfunctions of the kinetic term, with eigenvalues $$E_\textbf{k}=-\frac{1}{\mu\epsilon^2}\sum_i\left(\cos\left(\frac{2\pi}{N}k_i\right)-1\right).$$

In [2]:
function test_linearity(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        ψ, ϕ = rand(ComplexF64, shape), rand(ComplexF64, shape)
        α, β = rand(ComplexF64), rand(ComplexF64)
        n = lattice(N, length(shape))

        res = fill(1.0, (3,))

        for i in 1:3
            μ, ϵ = params[i]
            V = potential(μ, ϵ, n)
            δ = Hamiltonian(μ, ϵ, (@. α*ψ + β*ϕ), V) - α*Hamiltonian(μ, ϵ, ψ, V) - β*Hamiltonian(μ, ϵ, ϕ, V)
            res[i] = max(abs.(δ)...)
        end
        
        @printf "residues = [%.3e, %.3e, %.3e]" res...
    end
    
    println("Linearity:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

function test_hermiticity(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        ψ, ϕ = rand(ComplexF64, shape), rand(ComplexF64, shape)
        α, β = rand(ComplexF64), rand(ComplexF64)
        n = lattice(N, length(shape))

        res = fill(1.0, (3,))
        
        for i in 1:3
            μ, ϵ = params[i]
            V = potential(μ, ϵ, n)
            δ = dot(ψ, Hamiltonian(μ, ϵ, ϕ, V)) - dot(Hamiltonian(μ, ϵ, ψ, V), ϕ)
            res[i] = max(abs.(δ)...)
        end
        
        @printf "residues = [%.3e, %.3e, %.3e]" res...
    end
    
    println("Hermiticity:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

function test_positivity(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        ψ = rand(ComplexF64, shape)
        
        res = fill(1.0+im, (3,))
        
        for i in 1:3
            μ, ϵ = params[i]
            δ = dot(ψ, kinetic(μ, ϵ, ψ))
            res[i] = δ
        end
        
        print([real(res)[i] >= 0 for i in 1:3])
        @printf ", imaginary parts = [%.3e, %.3e, %.3e]" imag(res)...
    end
    
    println("Positivity:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

function test_eigenfunctions(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        k = rand(Int16, (length(shape)))
        K = fill(k, shape)
        n = lattice(N, length(shape))
        ψ_k = @. exp(2*pi*im/N * dot(n, K))

        res = fill(1.0, (3,))
        
        for i in 1:3
            μ, ϵ = params[i]
            δ = kinetic(μ, ϵ, ψ_k) + 1/(μ*ϵ^2) * (sum(cos.(2*pi/N * k)) - length(shape)) * ψ_k
            res[i] = max(abs.(δ)...)
        end
        
        @printf "residues = [%.3e, %.3e, %.3e]" res...
    end
    
    println("Eigenfunctions:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

test_eigenfunctions (generic function with 1 method)

In [3]:
params = [(1,1), (20,0.1), (0.1,20)]

println("(μ, ϵ) = ", string(params)[18:end], "\n")
test_linearity(10, params)
test_hermiticity(10, params)
test_positivity(10, params)
test_eigenfunctions(10, params)

(μ, ϵ) = [(1, 1), (20, 0.1), (0.1, 20)]



Linearity:
D=1: residues = [7.105e-15, 8.671e-16, 6.508e-11]


D=2: residues = [3.352e-14, 2.868e-15, 9.533e-10]


D=3: residues = [1.729e-13, 6.217e-15, 1.920e-09]


Hermiticity:

...

[1, 1, 1], imaginary parts = [0.000e+00, 2.220e-16, 1.735e-18]
D=2: Bool[1, 1, 1], imaginary parts = [-1.332e-15, -1.776e-15, 0.000e+00]
D=3: Bool[1, 1, 1], imaginary parts = [-9.770e-15, -3.197e-14, 9.714e-17]
Eigenfunctions:


D=1: 

residues = [1.032e-12, 5.158e-12, 2.579e-14]
D=2: 

residues = [9.015e-12, 4.507e-11, 2.254e-13]
D=3: 

residues = [2.003e-11, 1.002e-10, 5.008e-13]


The residues are consistently of order $10^{-6}$ or smaller for all tests. They tend to increase with the number of dimensions.

**Tests of the eigenvalue algorithms**

In [10]:
function test_power_method(N::Int, tol::Float64, maxiters::Int; positive_multiple::Number = 1000))
    
    function test_engine(shape::Tuple{Vararg{Int}})
        M = prod(shape)
        A = randn(ComplexF64, (M,M))
        A = A' * A + positive_multiple * I ; A = (A + A')/2
        
        function apply_A(v::Array)
            @assert size(v) == shape
            return reshape(A*reshape(v, (M,)), shape)
        end
        
        λ, v, niters = power_method(shape, apply_A, tol, maxiters)
        δ = apply_A(v) - λ*v
        res = max(abs.(δ)...)

        @printf "residue = %.3e" res
        print(", iterations = ", niters, ", Test passes: ", res <= tol)
    end
    
    println("Power method:")
    print("D=1: "); @time "; runtime" test_engine((N,))
    print("D=2: "); @time "; runtime" test_engine((N,N))
    print("D=3: "); @time "; runtime" test_engine((N,N,N)); print("\n")
end

function test_conjugate_gradient(N::Int, tol::Float64, maxiters::Int)
    
    function test_engine(shape::Tuple{Vararg{Int}})
        M = prod(shape)
        A = randn(ComplexF64, (M,M))
        A = A' * A; A = (A + A')/2 #make A positive definite & hermitian
        b = randn(ComplexF64, shape)
        
        function apply_A(v::Array)
            @assert size(v) == shape
            return reshape(A*reshape(v, (M,)), shape)
        end
        
        x, niters = conjugate_gradient(apply_A, b, tol, maxiters)
        δ = apply_A(x) - b
        res = norm(δ)

        @printf "residue = %.3e" res
        print(", iterations = ", niters, ", test passes: ", res <= tol*norm(b))
    end

    println("Conjugate Gradient:")
    print("D=1: "); @time "; runtime" test_engine((N,))
    print("D=2: "); @time "; runtime" test_engine((N,N))
    print("D=3: "); @time "; runtime" test_engine((N,N,N)); print("\n")
end

test_conjugate_gradient (generic function with 1 method)

In [11]:
test_power_method(8, 1e-10, 10000)
test_conjugate_gradient(8, 1e-10, 10000)

Power method:
D=1: residue = 0.000e+00, iterations = 1, Test passes: true; runtime: 0.010030 seconds (304 allocations: 18.883 KiB, 91.45% compilation time)
D=2: 

ErrorException: Maximum number of power method iterations reached

The algorithms converge quickly for small $N$, but the number of iterations and the runtime increase very quickly with $N$, especially for $D=3$.